In [1]:
#setup

import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from Hourbid import *
from blockbid import *
from Fibid import *

In [3]:
start_time = time.time()

df = pd.read_csv(r'C:\Users\evzqn\OneDrive\Documents\TURKISH DAM\Veri_4.in', header = None) # you have to change this to where you guys have the data set saved
N = df[0].tolist()
B = df[3].tolist() 
S = df[2].tolist()
Q = df[4].tolist()
P = df[5].tolist()
L = df[6].tolist()
pnt = df[7].tolist()
C_P = []
BR = []
FR = []
FD = []
FS = []
for i in range(0,len(B)):
    if B[i] == 'B':
        BR.append([P[i],B[i],S[i],Q[i],L[i], N[i], pnt[i]])
    if B[i] == 'F':
        FR.append([P[i],B[i],S[i],Q[i],L[i], N[i]])
    if B[i] == 'F' and Q[i] > 0 :
        FD.append(N[i])
    if B[i] == 'F' and Q[i] < 0 :
        FS.append(N[i])

iteration_number = 0
iteration_limit = 20
grand_iteration_number = 0
grand_iteration_limit = 10




acc = []
C_P = []
AX = []
for j in range(1,25):
    XD = []
    XS = []
    X = []
    for i in range(0,len(B)) :
        if B[i] == 'S' and S[i] == j:
            X.append([P[i]])
            if Q[i] > 0 :
                XD.append([P[i],Q[i]])
            if Q[i] < 0 :
                XS.append([P[i],Q[i]])
    X.sort()
    XD.sort()
    XS.sort()
    #print(XD)
    #print(XS)
    X[0].append(XD[0][1]+ XS[0][1])
    X[1].append(XD[0][1]+ XS[0][1])
    for i in range(2,len(X)-2):
        kd = 0
        ks = 0
        while XD[kd][0]<=X[i][0] :
            kd+=1
        while XS[ks][0]<=X[i][0] :
            ks+=1
        X[i].append(float(np.interp(X[i], (XD[kd-1][0],XD[kd][0]) ,(XD[kd-1][1],XD[kd][1])) + np.interp(X[i], (XS[ks-1][0],XS[ks][0]) ,(XS[ks-1][1],XS[ks][1]))))
    X[len(X)-1].append((XD[len(XD)-1][1]) + (XS[len(XS)-1][1]))
    X[len(X)-2].append((XD[len(XD)-1][1]) + (XS[len(XS)-1][1]))
    AX.append(X)
    a = 0
    b = len(X)-1
    while abs(b-a) != 1 :
        c = int(a + ((b-a)/2) + (((b-a)/2)%1))
        if X[c][1] >= 0 :
            a = c
        if X[c][1] < 0 :
            b = c
    C_P.append(np.interp(0, (X[b][1],X[a][1]),((X[b][0],X[a][0]))))
#print(C_P)   #intial clearing prices


FRS = []
FRD = []
FKS = []
FKD = []

for i in range (0,len(B)):
    if B[i] == 'F' and Q[i] < 0:
        FRS.append([P[i],Q[i],N[i],0])
    if B[i] == 'F' and Q[i] > 0:
        FRD.append([P[i],Q[i],N[i],0])

#incorporating block bids

BR = []
for i in range(0,len(B)):
    if B[i] == 'B':
        BR.append([P[i],B[i],S[i],Q[i],L[i], N[i], pnt[i]])
        
for i in range(0, len(BR)):
    C_P_avg = 0
    #print(BR[i][2]-1, BR[i][2]+BR[i][4])
    for j in range(BR[i-1][2]-1, BR[i-1][2]-1 +BR[i-1][4]):
        C_P_avg += C_P[j]
        
    C_P_avg = C_P_avg/(BR[i-1][4])
    #print(C_P_avg)
    BR[i-1].append((BR[i-1][0]-C_P_avg)*abs(BR[i-1][3])*(BR[i-1][4]))         #B Format = [P, B, S, Q, L, N, pnt]

for i in range(0, len(BR)):
    IV = BR[i][7]
    BR[i].remove(BR[i][7])
    BR[i].insert(0,IV)
        
BK = []
BKD = []
BKS = []
BPA = []
FK = []
FPA = []
BAN = []
HBK = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
HBKN = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
HBKP = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
BR.sort() # B Format = [IV, P, B, S, Q, L, N, pnt]
BR.reverse()
BK.append(BR[0])
BR.remove(BR[0])

while grand_iteration_number < grand_iteration_limit :
    while iteration_number < iteration_limit :
        for i in range(0, len(BK)-1):
            if BK[i][0] < 0 :
                BR.append(BK[i])
                BK.remove(BK[i])
        BR.sort()
        BR.reverse()
        iteration_number += 1
        while BK[0][0] and len(BR) != 0 > 0 :
            for j in range(BK[0][3]-1, BK[0][3]+BK[0][5]-1):
                HBK[j] = HBK[j] + BK[0][4]
                if BK[0][4] > 0 :
                    HBKP[j] = HBKP[j] + BK[0][4]
                    
                else:
                    HBKN[j] = HBKN[j] + BK[0][4]
            for j in range(0, 24):
                a = 0
                b = len(AX[j])-1
                while abs(b-a) != 1 :
                    c = int(a + ((b-a)/2) + (((b-a)/2)%1))
                    if AX[j][c][1] + HBK[j] >= 0 :
                        a = c
                    if AX[j][c][1] + HBK[j] < 0 :
                        b = c
                C_P[j] = (np.interp(0, (AX[j][b][1] + HBK[j], AX[j][a][1] + HBK[j]),((AX[j][b][0], AX[j][a][0]))))


            for i in range(0, len(BR)):
                C_P_avg = 0
                for j in range(BR[i-1][3]-1, BR[i-1][3]-1+BR[i-1][5]):
                    C_P_avg += C_P[j]
        
                C_P_avg = C_P_avg/(BR[i-1][5])
                
                BR[i-1][0] = ((BR[i-1][1]-C_P_avg)*abs(BR[i-1][4])*(BR[i-1][5]))
            BR.sort()
            BR.reverse()
            BK.append(BR[0])
            BR.remove(BR[0])
            BK.reverse()
    #print(C_P) #Clearing prices after block bids


    # Accounting for linked bids
    NlenK = len(BK) - 1 
    NlenR = len(BR) - 1
    i=0
    j=0
    while i < NlenK:
        while j < NlenR:
            if BR[j][6] == BK[i][7]:
                print([BR[j] , BK[i]])
                BR.append(BK[i])
                for k in range(BK[i][3]-1, BK[i][3]+BK[i][5]-1):
                    HBK[k] -= BK[i][4]
                    if BK[0][4] > 0 :
                        HBKP[k] = HBKP[k] - BK[0][4]
                    else:
                        HBKN[k] = HBKN[k] - BK[0][4]
                BK.remove(BK[i])
                NlenK = len(BK) - 1
                NlenR = len(BR) - 1
                for z in range(0, 24):
                    a = 0
                    b = len(AX[z])-1
                    while abs(b-a) != 1 :
                        c = int(a + ((b-a)/2) + (((b-a)/2)%1))
                        if AX[z][c][1] + HBK[z] >= 0 :
                            a = c
                        if AX[z][c][1] + HBK[z] < 0 :
                            b = c
                    C_P[z] = (np.interp(0, (AX[z][b][1] + HBK[z], AX[z][a][1] + HBK[z]),((AX[z][b][0], AX[z][a][0]))))
            j+=1
        i+=1

   # print(C_P) # clearing prices after accounting for linked bids

    #Incorporating Flex Bids


    FRS.sort()
    FRD.sort()
    FRD.reverse()

    C_Pmin = 10000000
    C_Pmax = 0

    for k in range(1,25):
        if C_P[k-1] < C_Pmin:
            C_Pmin = C_P[k-1]
            Smin = k
        if C_P[k-1] > C_Pmax:
            C_Pmax = C_P[k-1]
            Smax = k



    while FRD != [] and FRD[0][0] > C_Pmin :
        FKD.append(FRD[0])
        FKD[-1][3] = Smin
        HBK[Smin-1] = HBK[Smin-1] + FRD[0][1]
        if FRD[0][1] > 0 :
            HBKP[Smin-1] = HBKP[Smin-1] + FRD[0][1]
        else:
            HBKN[Smin-1] = HBKN[Smin-1] + FRD[0][1]
        FRD.remove(FRD[0])
        FRD.sort()
        FRD.reverse()    
        a = 0
        b = len(AX[Smin-1])-1
        while abs(b-a) != 1 :
            c = int(a + ((b-a)/2) + (((b-a)/2)%1))
            if AX[Smin-1][c][1] + HBK[Smin-1] > 0 :
                a = c
            if AX[Smin-1][c][1] + HBK[Smin-1] < 0 :
                b = c
        C_P[Smin-1] = (np.interp(0, (AX[Smin-1][b][1] + HBK[Smin-1], AX[Smin-1][a][1] + HBK[Smin-1]),((AX[Smin-1][b][0], AX[Smin-1][a][0]))))
        C_Pmin = 10000000
        C_Pmax = 0    
        for k in range(1,25):

            if C_P[k-1] < C_Pmin:
                C_Pmin = C_P[k-1]
                Smin = k
            if C_P[k-1] > C_Pmax:
                C_Pmax = C_P[k-1]
                Smax = k

    while FRS != [] and FRS[0][0] < C_Pmax :
        FKS.append(FRS[0])
        FKS[-1][3] = Smax
        HBK[Smax-1] = HBK[Smax-1] + FRS[0][1]
        if FRS[0][1] > 0 :
            HBKP[Smax-1] = HBKP[Smax-1] + FRS[0][1]
        else:
            HBKN[Smax-1] = HBKN[Smax-1] + FRS[0][1]
        FRS.remove(FRS[0])
        FRS.sort()    
        a = 0
        b = len(AX[Smax-1])-1
        while abs(b-a) != 1 :
            c = int(a + ((b-a)/2) + (((b-a)/2)%1))
            if AX[Smax-1][c][1] + HBK[Smax-1] >= 0 :
                a = c
            if AX[Smax-1][c][1] + HBK[Smax-1] < 0 :
                b = c
        C_P[Smax-1] = (np.interp(0, (AX[Smax-1][b][1] + HBK[Smax-1], AX[Smax-1][a][1] + HBK[Smax-1]),((AX[Smax-1][b][0], AX[Smax-1][a][0]))))
        C_Pmin = 10000000
        C_Pmax = 0    
        for k in range(1,25):

            if C_P[k-1] < C_Pmin:
                C_Pmin = C_P[k-1]
                Smin = k
            if C_P[k-1] > C_Pmax:
                C_Pmax = C_P[k-1]
                Smax = k


    grand_iteration_number += 1
   # print(C_P) #Clearing Prices After Flex Bids

#Feasibility check
NlenR = len(BR)
i = 0
while i < NlenR : 
    if BR[i][0] > 0 :
        if BR[i][7] == 'NaN':
            print(BR[i])
            BK.append(BR[i])
            BK.reverse()
            BR.remove(BR[i])
            for j in range(BK[0][3]-1, BK[0][3]+BK[0][5]-1):
                HBK[j] = HBK[j] + BK[0][4]
                if BK[0][4] > 0 :
                    HBKP[j] = HBKP[j] + BK[0][4]
                else:
                    HBKN[j] = HBKN[j] + BK[0][4]
            NlenR = len(BR)
                
            for j in range(0, 24):
                a = 0
                b = len(AX[j])-1
                while abs(b-a) != 1 :
                    c = int(a + ((b-a)/2) + (((b-a)/2)%1))
                    if AX[j][c][1] + HBK[j] > 0 :
                        a = c
                    if AX[j][c][1] + HBK[j] < 0 :
                        b = c
                C_P[j] = (np.interp(0, (AX[j][b][1] + HBK[j], AX[j][a][1] + HBK[j]),((AX[j][b][0], AX[j][a][0]))))

            for k in range(0, len(BR)):
                C_P_avg = 0
                for j in range(BR[k-1][3]-1, BR[k-1][3]+BR[k-1][5]):
                    C_P_avg += C_P[j-1]
        
                C_P_avg = C_P_avg/(BR[k-1][5])
                
                BR[k-1][0] = ((BR[k-1][1]-C_P_avg)*abs(BR[k-1][4])*(BR[k-1][5])) #[IV, P, B, S, Q, L, N, pnt]

        for j in range(0,len(BK)):
            if BR[i][7] == BK[j][6] :
                print(BR[i])
                BK.append(BR[i])
                BR.remove(BR[i])
                BK.reverse()
                for j in range(BK[0][3]-1, BK[0][3]+BK[0][5]-1):
                    HBK[j] = HBK[j] + BK[0][4]
                    if BK[0][4] > 0 :
                        HBKP[j] = HBKP[j] + BK[0][4]
                    else:
                        HBKN[j] = HBKN[j] + BK[0][4]
                NlenR = len(BR) 
                for j in range(0, 24):
                    a = 0
                    b = len(AX[j])-1
                    while abs(b-a) != 1 :
                        c = int(a + ((b-a)/2) + (((b-a)/2)%1))
                        if AX[j][c][1] + HBK[j] > 0 :
                            a = c
                        if AX[j][c][1] + HBK[j] < 0 :
                            b = c
                    C_P[j] = (np.interp(0, (AX[j][b][1] + HBK[j], AX[j][a][1] + HBK[j]),((AX[j][b][0], AX[j][a][0]))))

                for k in range(0, len(BR)):
                    C_P_avg = 0
                    for j in range(BR[k-1][3]-1, BR[k-1][3]-1+BR[k-1][5]):
                        C_P_avg += C_P[j]
            
                    C_P_avg = C_P_avg/(BR[k-1][5])
                    
                    BR[k-1][0] = ((BR[k-1][1]-C_P_avg)*abs(BR[k-1][4])*(BR[k-1][5])) #[IV, P, B, S, Q, L, N, pnt]
    i+=1
print(C_P) # final clearing prices
end_time = time.time() - start_time
print(end_time) # CPU time it took to run the program
print(FKD)
print(FKS)
print(BK)


# HD HS BD BS FD FS
solvarBD = []
solvarBS = []
solvarFD = []
solvarFS = []
for i in range(0,len(BK)):
    acc.append(BK[i][6])

for i in range(0,len(FKD)):
    acc.append([FKD[i][2], FKD[i][3]])

for i in range(0,len(FKS)):
    acc.append([FKS[i][2], FKS[i][3]])

hourlen = 0
blocklen = 0
flexlen = 0
flexlenD = 0
flexlenS = 0
for i in range(0,len(B)):
    if B[i] == 'S':
        hourlen+=1
    if B[i] == 'B':
        blocklen+=1
    if B[i] == 'F'and Q[i] > 0:
        flexlenD+=1
        flexlen +=1
    if B[i] == 'F'and Q[i] < 0:
        flexlenS+=1
        flexlen +=1

for i in range(hourlen, hourlen + blocklen):
    if N[i] in acc and Q[i] > 0:
        solvarBD.append(1)
    if N[i] not in acc and Q[i] > 0:
        solvarBD.append(0)
    if N[i] in acc and Q[i] < 0:
        solvarBS.append(1)
    if N[i] not in acc and Q[i] < 0:
        solvarBS.append(0)
AFacc = []
FaccD = []
NFaccD = []
FaccDH = []
NFaccDH = []
FaccS = []
NFaccS = []
FaccSH = []
NFaccSH = []
for i in range(0,len(FKD)):
    FaccD.append(FKD[i][2])
    FaccDH.append(FKD[i][3])

for i in range(0,len(B)):
    for j in range(0,len(FaccD)):
        if N[i] == FaccD[j] :
            NFaccD.append(N[i])
            NFaccDH.append(FaccDH[j])

for i in range(0,len(FKS)):
    FaccS.append(FKS[i][2])
    FaccSH.append(FKS[i][3])


for i in range(0,len(B)):
    for j in range(0,len(FaccS)):
        if N[i] == FaccS[j] :
            NFaccS.append(N[i])
            NFaccSH.append(FaccSH[j])

for i in range(0,24):
    AFacc.append([[0]*flexlenD,[0]*flexlenS])

idx=0
indx=0
for i in range(hourlen + blocklen, hourlen + blocklen + flexlen):
    if N[i] in NFaccD:
        for j in range(0,len(NFaccD)):
            for k in range(0,len(FD)):
                if N[i] == NFaccD[j] and N[i] == FD[k]:
                    hour = j
                    idx = k
        AFacc[NFaccDH[hour]-1][0][idx] = 1
    if N[i] in NFaccS:
        for j in range(0,len(NFaccS)):
            for k in range(0,len(FS)):
                if N[i] == NFaccS[j] and N[i] == FS[k]:
                    hour = j
                    idx = k
        AFacc[NFaccSH[hour]-1][1][idx] = 1
print(solvarBD)
print(solvarBS)
NAFacc = [[],[]]
for i in range(0,24):
    NAFacc[0].append(AFacc[i][0])
    NAFacc[1].append(AFacc[i][1])
print(NAFacc)

HS = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
HD = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
for j in range(0,24):
    for i in range(0,hourlen):
        if S[i] == j+1 and Q[i] > 0 and P[i] >= C_P[j]:
            HD[j].append([1, P[i]])
        if S[i] == j+1 and Q[i] > 0 and P[i] < C_P[j]:
            HD[j].append([0, P[i]])
        if S[i] == j+1 and Q[i] < 0 and P[i] <= C_P[j]:
            HS[j].append([1, P[i]])
        if S[i] == j+1 and Q[i] < 0 and P[i] > C_P[j]:
            HS[j].append([0, P[i]])

for j in range(0,24):
    for i in range(0,len(HD[j])-1):
        if [HD[j][i][0],HD[j][i+1][0]] == [0,1]:
            HD[j][i+1][0] = ((HD[j][i][1] - C_P[j])/(HD[j][i][1]-HD[j][i+1][1]))
    for i in range(0,len(HS[j])-1):
        if [HS[j][i][0],HS[j][i+1][0]] == [1,0]:
            HS[j][i][0] = ((C_P[j]-HS[j][i][1])/(HS[j][i+1][1]-HS[j][i][1]))

for j in range(0,24):
    for i in range(0,len(HD[j])):
        HD[j][i] = HD[j][i][0]
    for i in range(0,len(HS[j])):
        HS[j][i] = HS[j][i][0]


for j in range(0,24):
    HD[j].pop(0)
    HS[j].pop(0)
#print(((HS)))
#print(((HD)))



Hour_D = HD
Hour_S = HS
Flex_D = [NAFacc[0]]
Flex_D = Flex_D[0]
Flex_S = [NAFacc[1]]
Flex_S = Flex_S[0]
H_S,H_D,x=HourBid()
BS_bids,BD_bids = Blockbid()
FS_bids,FD_bids = Fibid()
HD_bids_P=H_D['price']
HD_bids_Q=H_D['Quantity']
HS_bids_P=H_S['price']
HS_bids_Q=H_S['Quantity']
Block_D = solvarBD
Block_S = solvarBS
HDP = []
HDQ = []
for i in range(len(H_D)):
    HDP.append(H_D['price'][len(H_D)-i -1])
    HDQ.append(H_D['Quantity'][len(H_D)-i -1])
#print(len(Block_S))
#print(BS_bids)

Res_HS = 0
for j in range(0,24) :
    i = 0
    while HS_bids_P[1000*j + i] < C_P[j] : 
        Res_HS += ((HS_bids_Q[1000*j + i + 1] - HS_bids_Q[1000*j + i])*((HS_bids_P[1000*j + i + 1] + HS_bids_P[1000*j + i])/2))
        #print((HS_bids_Q[1000*j + i + 1] - HS_bids_Q[1000*j + i])*((HS_bids_P[1000*j + i + 1] + HS_bids_P[1000*j + i])/2))
        i+=1
    Res_HS += ((C_P[j] - HS_bids_P[1000*j + i])/(HS_bids_P[1000*j + i + 1] - HS_bids_P[1000*j + i]))*((HS_bids_Q[1000*j + i + 1] - HS_bids_Q[1000*j + i])*((HS_bids_P[1000*j + i + 1] + HS_bids_P[1000*j + i])/2))
print(Res_HS)

Res_HD = 0
for j in range(0,24) :
    i = 0
    while HD_bids_P[1000*j + i] > C_P[j] : 
        Res_HD += (HD_bids_Q[1000*j + i + 1] - HD_bids_Q[1000*j + i])*((HD_bids_P[1000*j + i + 1] + HD_bids_P[1000*j + i])/2)
        i+=1
    Res_HD -= ((HD_bids_P[1000*j + i] - C_P[j])/((HD_bids_P[1000*j + i] - HD_bids_P[1000*j + i + 1])))*(HD_bids_Q[1000*j + i + 1] - HD_bids_Q[1000*j + i])*((HD_bids_P[1000*j + i + 1] + HD_bids_P[1000*j + i])/2)

print(Res_HD)

Res_BD = sum(BD_bids['Duration'][i]*BD_bids['Quantity'][i]*BD_bids['price'][i]*Block_D[i] for i in range(len(Block_D)))
print(Res_BD)
Res_BS = sum(BS_bids['Duration'][i]*BS_bids['Quantity'][i]*BS_bids['price'][i]*Block_S[i] for i in range(len(Block_S)))
print(Res_BS)
    
Res_FD = sum(FD_bids['Quantity'][i]*FD_bids['price'][i]*sum(Flex_D[h][i] for h in range(24)) for i in range(len(FD_bids))) 
print(Res_FD)
Res_FS = sum(FS_bids['Quantity'][i]*FS_bids['price'][i]*sum(Flex_S[h][i] for h in range(24)) for i in range(len(FS_bids)))
print(Res_FS)
minmax = 0 
for j in range(0,24):
    minmax += (HD_bids_Q[1000*j] - HD_bids_P[1000*j])
    minmax += (HS_bids_Q[1000*j] - HD_bids_P[1000*j])
print(minmax)

print(Res_HD - Res_HS + Res_BD - Res_BS + Res_FD - Res_FS + minmax)

KeyboardInterrupt: 